<a href="https://colab.research.google.com/github/dnevo/Practicum/blob/master/S17_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 50)
pd.set_option('display.width', 200)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:11,.2f}'.format
np.set_printoptions(precision=2)

In [ ]:
colab = True
if colab:
    data_path = 'https://raw.githubusercontent.com/dnevo/Practicum/master/datasets/'
else:
    data_path = '/datasets/'
df_phone = pd.read_csv(data_path + 'phone.csv')
df_personal = pd.read_csv(data_path + 'personal.csv')
df_internet = pd.read_csv(data_path + 'internet.csv')
df_contract = pd.read_csv(data_path + 'contract.csv')